In [ ]:
from board_detction import get_squares
from get_position import get_fen_from_board
import matplotlib.pyplot as plt
from comment import explain_move
# from code_Arm import move_piece,cleanup,get_pict
import tensorflow as tf 
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import pickle
import torch
from torchvision import transforms
from PIL import Image
import cv2
import chess
import chess.engine
import os
import numpy as np
import datetime
import chess
from tensorflow.keras.models import load_model



#-------------------------------------------------------------------------------------------------------------------------------------------   


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
COUNTER_FILE = "game_counter.txt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

engine_path=r"stockfish\stockfish-windows-x86-64-avx2.exe"
engine = chess.engine.SimpleEngine.popen_uci(engine_path)

model_for_piece = torch.load('models\chess_piece_classifier.pth')  # Load the piece classification model
model_for_piece = model_for_piece.to(DEVICE)  # Move model to the specified device

# Load the color model
model_for_color = torch.load('models\chess_color_classifier.pth')  # Load the color classification model
model_for_color = model_for_color.to(DEVICE)  

# Load the model
model = load_model("models based on previous games\chess_move_predictor.h5")

# Load the tokenizer
with open("models based on previous games\tokenizer.pkl", "rb") as file:
    tokenizer = pickle.load(file)


piece_comments = {
    "knight": 
        "Knights are tricky. Think carefully before deciding on their next step.",
    "queen": 
        "The queen's strength comes from her safety. Think twice before moving her.",
    "pawn": 
        "Every pawn step matters. Could any move from it create unexpected risks?",
    "rook": 
        "Your rook can dominate when placed wisely. Consider its role in your strategy.",
    "bishop":
        "A misplaced bishop can lose its power.be cearful if you want to move it",
    "king":
        "Your king is the cornerstone of your defense. Consider its safety carefully.",
}

col={x:i for i,x in enumerate(["a","b","c","d","e","f","g","h"])}
row={i+1:i for i in range(9)}

#-------------------------------------------------------------------------------------------------------------------------------------------    

def get_moved_piece(predicted_move, board):
    # Start Stockfish engine
    with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
        # Make the predicted move on the board
        move = chess.Move.from_uci(predicted_move)
        board.push(move)

        # Get the piece that moved
        moved_piece = board.piece_at(move.from_square)

        # Map the piece to a human-readable form
        piece_map = {
            chess.PAWN: "Pawn",
            chess.KNIGHT: "Knight",
            chess.BISHOP: "Bishop",
            chess.ROOK: "Rook",
            chess.QUEEN: "Queen",
            chess.KING: "King"
        }
        
        # Get the piece type that was moved
        moved_piece_type = piece_map.get(moved_piece.piece_type, "Unknown Piece")
        
        return moved_piece_type
    



def get_move(fen, level="beginner"):
    board = chess.Board(fen)
    
    # Set options based on player level
    if level == "beginner":
        options = {
            "Skill Level": 5,          
            "UCI_LimitStrength": True, # Limit engine strength
            "UCI_Elo": 1350           
        }
        time_limit = 0.1  
    elif level == "pro":
        options = {
            "Skill Level": 15,         
            "UCI_LimitStrength": False # Full engine strength
        }
        time_limit = 0.5  

    # Apply the options to Stockfish
    for option, value in options.items():
        engine.configure({option: value})

    # Get the best move within the time limit
    result = engine.play(board, chess.engine.Limit(time=time_limit))
    return result.move


def check_for_blunder(predicted_move, board):
    # Check if the predicted move is a blunder using Stockfish
    with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
        # Copy the current board state to evaluate before the move
        board_copy = board.copy()
        
        # Make the predicted move on the copied board
        move = chess.Move.from_uci(predicted_move)
        board_copy.push(move)
        
        # Evaluate the position after the move on the copied board
        evaluation = engine.analyse(board_copy, chess.engine.Limit(time=1.0))['score']
        
        # If the evaluation is worse than a certain threshold, consider it a blunder
        if evaluation.relative.score() < -100:
            return True
        return False




def get_move_from_fen(fen_before, fen_after):
    board_before = chess.Board(fen_before)
    board_after = chess.Board(fen_after)
    
    # Get all legal moves from the "before" board
    legal_moves = list(board_before.legal_moves)
    
    # Iterate through the legal moves and apply them to the board
    for move in legal_moves:
        board_before.push(move)
        # Compare only the piece placement part of the FEN (ignoring castling rights, move counters, etc.)
        #print(board_before.board_fen(),"----",board_after.board_fen())
        if board_before.board_fen() == board_after.board_fen():
            return move
        # Revert the move if it doesn't match
        board_before.pop()
        
    
    return None  # If no move matches, return None

        

def load_counter():
    if os.path.exists(COUNTER_FILE):
        with open(COUNTER_FILE, 'r') as f:
            return int(f.read().strip())
    return 0  # Start from 0 if the counter file doesn't exist

def save_counter(counter):
    with open(COUNTER_FILE, 'w') as f:
        f.write(str(counter))



# Check if the model file exists
# if os.path.exists(model_path):
#     model = ChessMovePredictor()  
#     # Load the model
#     model = torch.load(model_path)

def speak(msg):
    print(msg)     

class ChessGame:
    def __init__(self):
        self.board = chess.Board()
        self.active_color = 'White'
        self.moves = []
        self.result = None
        self.student_color ="White"
        self.arm_color ="Black"

        self.middle_game_start = False  # Flag for middle game start

    def make_move(self, move):
        if isinstance(move, chess.Move) and move in self.board.legal_moves:
            self.board.push(move)
            self.moves.append(move)   
            return True
        else:
            print("Illegal move!")
            return False


    def save_to_pgn(self, filename="games.pgn"):
        round=load_counter()+1
        save_counter(round)
        
        game = chess.pgn.Game()
        game.headers["Event"] = "TSPY12"
        game.headers["Site"] = "local"
        game.headers["Date"] = datetime.date.today().strftime("%Y.%m.%d")
        game.headers["Round"] = str(round)
        game.headers["White"] = "player" if self.student_color=="White" else "arm"
        game.headers["Black"] = "player" if self.student_color=="Black" else "arm"
        game.headers["Result"] = self.result or "*"


        # Add moves to PGN
        node = game
        for move in self.moves:
            node = node.add_variation(move)

        # Save to file
        with open(filename,"a") as pgn_file:
            print(game, file=pgn_file)

        
        

# Example usage
def play():
    pos=""
    game = ChessGame()
    game.student_color=input("what color will you play: " )
    game.arm_color=list({"White","Black"}-{game.student_color})[0]
    player_moves=[r"images\2-c5.JPG",r"images\4-d6.JPG",r"images\6-c4d5.JPG",r"images\8-kg6.JPG"]
    fen_before=game.board.fen()
    # while not game.board.is_game_over():
    for i in range(9):
        if game.active_color == game.student_color:
            if i%2==1:
                image=player_moves[i//2] 
            image = cv2.imread(image) #image=get_pict()

            squares=get_squares(image)

            fen_after=get_fen_from_board(squares,model_for_piece,model_for_color)

            move=get_move_from_fen(fen_before,fen_after)

            pos+=str(ai_move)+" "

            game.make_move(move)
            comment=explain_move(fen_before,fen_after, move)
            fen_before=game.board.fen()
            speak(comment) #speaker code
            game.active_color = 'White'
        else:
            ai_move = get_move(fen_before)  
            if i==0:
                ai_move=chess.Move.from_uci("e2e4")
            elif i==2:
                ai_move=chess.Move.from_uci("g1f3")    
            elif i==4:
                  ai_move=chess.Move.from_uci("d2d4") 
            elif i==6:
                ai_move=chess.Move.from_uci("f3d4")
            elif i==8:
                ai_move=chess.Move.from_uci("b1c3")    
            

            #____________________________________________________________________________________________________

            #____________________________________________________________________________________________________

            board_before = chess.Board(fen_before)
            piece =  board_before.piece_at(ai_move.from_square)

            moveToMake=str(ai_move)#for the next line
            # move_piece(col[moveToMake[0]],row[moveToMake[1]],col[moveToMake[2]],row[moveToMake[3]])

            speak(f"i played {piece} to {ai_move.uci()[2:]}") 
            game.make_move(ai_move)
            try:
                pos+=str(ai_move)+" "

                input_moves = pos.split()
                input_seq = tokenizer.texts_to_sequences([" ".join(input_moves)])[0]
                input_seq = pad_sequences([input_seq], maxlen=5)  # Adjust sequence_length if needed

                # Predict the next move
                predicted = model.predict(input_seq)
                predicted_move_index = np.argmax(predicted)
                predicted_move = tokenizer.index_word[predicted_move_index]
                moved_piece=predicted_move, game.board
                if check_for_blunder(predicted_move,game.board):
                    speak(comment[moved_piece])
            except:
                pass        
            
            fen_before = game.board.fen()
            game.active_color = 'Black' 

     # Announce game result
    if game.board.is_checkmate():
        speak("Checkmate! Game over.")
        if game.board.turn == chess.WHITE:
            speak("Black wins!")
            game.result="0-1"
        else:
            speak("White wins!")
            game.result="1-0"
    elif game.board.is_stalemate():
        speak("Stalemate! Game over.")    
        game.result="1/2-1/2"
        

    # save_game_to_pgn(game, filename="output_games.pgn")
    return game

game=play()


i played P to e4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
You moved your P from c7 to c5. Solid move, but there might have been a stronger option. Controlling the center with pawns is a key opening principle.
i played N to f3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
You moved your P from d7 to d6. This move controls the center, a key strategy in chess. Good move!
i played P to d4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
You moved your P from c5 to d4. This move controls the center, a key strategy in chess. Good move! You've played 1. d4, leading to closed games like the Queen's Gambit or King's Indian Defense.
i played N to d4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
You moved your N from g8 to f6. Knights are tricky pieces. Consider whether this move creates a tactical advantage, like a fork. Good move! Developing your minor pieces (knights and bishops) early helps control the center and prepares for castling.
i played N to c3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [ ]:
game.save_to_pgn("new.pgn")